In [5]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np

# データの読み込み
modelling_table = pd.read_csv("preprocessed_data.csv")

# valid_datetimeをdatetime型に変換
modelling_table['valid_datetime'] = pd.to_datetime(modelling_table['valid_datetime'])

# 季節ごとのデータフレームを作成
modelling_table['month'] = modelling_table['valid_datetime'].dt.month
spring = modelling_table[modelling_table['month'].isin([3, 4, 5])]
summer = modelling_table[modelling_table['month'].isin([6, 7, 8])]
autumn = modelling_table[modelling_table['month'].isin([9, 10, 11])]
winter = modelling_table[modelling_table['month'].isin([12, 1, 2])]

# 季節ごとの太陽光発電モデル
solar_models = {
    'spring': smf.quantreg('Solar_MWh_credit ~ bs(Radiation_dwd,df=5) + bs(Radiation_ncep,df=5)', data=spring),
    'summer': smf.quantreg('Solar_MWh_credit ~ bs(Radiation_dwd,df=5) + bs(Radiation_ncep,df=5)', data=summer),
    'autumn': smf.quantreg('Solar_MWh_credit ~ bs(Radiation_dwd,df=5) + bs(Radiation_ncep,df=5)', data=autumn),
    'winter': smf.quantreg('Solar_MWh_credit ~ bs(Radiation_dwd,df=5) + bs(Radiation_ncep,df=5)', data=winter)
}

# 風力発電モデル
wind_mod = smf.quantreg('Wind_MWh_credit ~ bs(WindSpeed_dwd,df=8) + bs(WindSpeed_ncep,df=8)', data=modelling_table)

forecast_models_solar = dict()
forecast_models_wind = dict()

# 季節ごとの太陽光発電と風力発電の分位数を計算
for quantile in range(10, 100, 10):
    for season, season_months in {'spring': [3, 4, 5], 'summer': [6, 7, 8], 'autumn': [9, 10, 11], 'winter': [12, 1, 2]}.items():
        forecast_models_solar[f"{season}_q{quantile}"] = solar_models[season].fit(q=quantile/100, max_iter=2500)
        modelling_table.loc[modelling_table['month'].isin(season_months), f"{season}_q{quantile}_solar"] = forecast_models_solar[f"{season}_q{quantile}"].predict(modelling_table.loc[modelling_table['month'].isin(season_months)])
    
    forecast_models_wind[f"q{quantile}"] = wind_mod.fit(q=quantile/100, max_iter=2500)
    modelling_table[f"q{quantile}_wind"] = forecast_models_wind[f"q{quantile}"].predict(modelling_table)

    # 発電量は0以上、小数点3桁で丸める
    for season in solar_models:
        modelling_table.loc[modelling_table[f"{season}_q{quantile}_solar"] < 0, f"{season}_q{quantile}_solar"] = 0
        modelling_table[f"{season}_q{quantile}_solar"] = modelling_table[f"{season}_q{quantile}_solar"].round(2)
    
    modelling_table.loc[modelling_table[f"q{quantile}_wind"] < 0, f"q{quantile}_wind"] = 0
    modelling_table[f"q{quantile}_wind"] = modelling_table[f"q{quantile}_wind"].round(2)

# 太陽光と風力の発電量の合計をトータル発電量として計算
for quantile in range(10, 100, 10):
    for season in solar_models:
        modelling_table[f"{season}_q{quantile}"] = modelling_table[f"{season}_q{quantile}_solar"] + modelling_table[f"q{quantile}_wind"]

# 更新データを保存
modelling_table.to_csv("analyzed_data_sep_sea.csv", index=False)

# モデルをファイルに保存
for quantile in range(10,100,10):
    for season in solar_models:
        forecast_models_solar[f"{season}_q{quantile}"].save(f"models_sep_sea/{season}_solar_model_q{quantile}.pickle")
    #forecast_models_wind[f"q{quantile}"].save(f"models_sep/wind_model_q{quantile}.pickle") # 風力のモデルはmodels_sepに存在

/usr/local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (2500) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/usr/local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (2500) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/usr/local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (2500) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


In [7]:
modelling_table.columns

Index(['ref_datetime', 'valid_datetime', 'WindSpeed_dwd', 'WindSpeed_ncep',
       'Radiation_dwd', 'Radiation_ncep', 'Wind_MWh_credit',
       'Solar_MWh_credit', 'total_generation_MWh', 'month', 'spring_q10_solar',
       'summer_q10_solar', 'autumn_q10_solar', 'winter_q10_solar', 'q10_wind',
       'spring_q20_solar', 'summer_q20_solar', 'autumn_q20_solar',
       'winter_q20_solar', 'q20_wind', 'spring_q30_solar', 'summer_q30_solar',
       'autumn_q30_solar', 'winter_q30_solar', 'q30_wind', 'spring_q40_solar',
       'summer_q40_solar', 'autumn_q40_solar', 'winter_q40_solar', 'q40_wind',
       'spring_q50_solar', 'summer_q50_solar', 'autumn_q50_solar',
       'winter_q50_solar', 'q50_wind', 'spring_q60_solar', 'summer_q60_solar',
       'autumn_q60_solar', 'winter_q60_solar', 'q60_wind', 'spring_q70_solar',
       'summer_q70_solar', 'autumn_q70_solar', 'winter_q70_solar', 'q70_wind',
       'spring_q80_solar', 'summer_q80_solar', 'autumn_q80_solar',
       'winter_q80_solar', 'q

In [8]:
modelling_table.head()

,ref_datetime,valid_datetime,WindSpeed_dwd,WindSpeed_ncep,Radiation_dwd,Radiation_ncep,Wind_MWh_credit,Solar_MWh_credit,total_generation_MWh,month,...,autumn_q70,winter_q70,spring_q80,summer_q80,autumn_q80,winter_q80,spring_q90,summer_q90,autumn_q90,winter_q90
0,2020-09-20 00:00:00+00:00,2020-09-20 00:00:00+00:00,11.802604,11.338991,0.0,0.0,498.142,0.0,498.142,9,...,530.79,NaN,NaN,NaN,547.02,NaN,NaN,NaN,565.93,NaN
1,2020-09-20 00:00:00+00:00,2020-09-20 00:30:00+00:00,11.648818,11.516161,0.0,0.0,478.788,0.0,478.788,9,...,533.36,NaN,NaN,NaN,549.20,NaN,NaN,NaN,567.39,NaN
2,2020-09-20 00:00:00+00:00,2020-09-20 01:00:00+00:00,11.495032,11.693331,0.0,0.0,470.522,0.0,470.522,9,...,534.88,NaN,NaN,NaN,550.50,NaN,NaN,NaN,568.24,NaN
3,2020-09-20 00:00:00+00:00,2020-09-20 01:30:00+00:00,11.354128,11.716686,0.0,0.0,482.183,0.0,482.183,9,...,532.07,NaN,NaN,NaN,548.23,NaN,NaN,NaN,566.69,NaN
4,2020-09-20 00:00:00+00:00,2020-09-20 02:00:00+00:00,11.213223,11.740043,0.0,0.0,459.216,0.0,459.216,9,...,528.90,NaN,NaN,NaN,545.65,NaN,NaN,NaN,564.92,NaN
